In [ ]:
from collections import defaultdict
from gensim import corpora

documents = [
    "Human machine social interface for lab abc computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human social system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
    "Pocahontas is a social human who likes interaction",
    "There was a shooting in alabama"
]

# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [
    [word for word in document.lower().split() if word not in stoplist]
    for document in documents
]

# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    #print(f"ON {text}...")
    for token in text:
        #print(token)
        frequency[token] += 1 # frequency count

# list of unique words per document in corpus
texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

# returns gensim object of dictionary mapping
dictionary = corpora.Dictionary(texts)

# list of tuple(token_id, token_count)
corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
[text for text in texts]

In [ ]:
# also dictionary.token2id()
print(dictionary.keys())
print([dictionary[i] for i in dictionary.keys()])

In [ ]:
# (token_id, token_count)
corpus

In [ ]:
# Contents of dictionary
print(f"Contents of dictionary: {[dictionary[i] for i in dictionary.keys()]}")

In [ ]:
# Create LSI model using dictionary as training data
from gensim import models
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=1)

In [ ]:
# Output most meaningful words in corpus
lsi.print_topics()

In [ ]:
#test_doc = "Girard is a social science student interested in computer science and human computer interaction and natural language processing"
#test_doc = "Girard is a mechanic and a mountaineer that likes to cook."

# Add last sentence to bring similarity score up
test_doc = "We started off early, but by the time we got a glimpse of the Aberdeen glacier, there were two parties midway through the ice pitch already. We could barely make them out from a far, and it seemed like the wall was not that much of a deal. It was pretty daunting once you get close enought to appreciate the massive ice formation."

In [ ]:
vec_bow = dictionary.doc2bow(test_doc.lower().split())
vec_lsi = lsi[vec_bow]  # convert the query test_doc to LSI matrix and calculate cosine similarity against the lsi model
print(vec_lsi)

In [ ]:
from gensim import similarities
index = similarities.MatrixSimilarity(lsi[corpus])

index.get_similarities(test_doc, vec_lsi)

index.save('/tmp/deerwester.index')
index = similarities.MatrixSimilarity.load('/tmp/deerwester.index')

In [ ]:
documents

In [ ]:
sims = index[vec_lsi]  # perform a similarity query against the corpus
print(list(enumerate(sims)))  # print (document_number, document_similarity) 2-tuples

In [ ]:
sims = sorted(enumerate(sims), key=lambda item: -item[1])
for doc_position, doc_score in sims:
    print(doc_score, documents[doc_position])

In [ ]:
sims

In [ ]:
type(documents)